In [ ]:
import os
from diffaux.disk_bulge_modeling.generate_bulge_disk_sample import (
    get_bulge_disk_test_sample,
    get_bulge_disk_decomposition,
    get_zindexes,
)
from jax import random as jran
import numpy as np
import jax.numpy as jnp
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from itertools import zip_longest

ran_key = jran.key(0)

In [ ]:
halo_key, ran_key = jran.split(ran_key, 2)
lgmp_min = 11.0
redshift = 0.05
Lbox = 100.0
diffstar_cens = get_bulge_disk_test_sample(halo_key, lgmp_min=lgmp_min, redshift=redshift, Lbox=Lbox)

## logsm0 Bug-fixed Version

In [ ]:
disk_bulge_key, ran_key = jran.split(ran_key, 2)
diffstar_cens = get_bulge_disk_decomposition(disk_bulge_key, diffstar_cens, new_model=False)
print(diffstar_cens.keys())
for q in ['tcrit_bulge', 'fbulge_early', 'fbulge_late']:
    print(q, diffstar_cens[q].shape)

In [ ]:
from dsps.constants import SFR_MIN
from diffaux.disk_bulge_modeling.disk_bulge_kernels import (
    calc_tform_pop,
    _bulge_fraction_vs_tform,
    _bulge_fraction_kernel,
)
from diffaux.validation.plot_utilities import get_nrow_ncol, get_subsample

zvalues = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
redshifts = diffstar_cens["z_table"]
zindexes, zs = get_zindexes(zvalues, redshifts)

from diffaux.validation.plot_disk_bulge import (
    plot_qs_nocuts,
    plot_histories,
    plot_q_with_cuts,
    plot_q1_q2,
    plot_q1_vs_q2,
    plot_q_vs_q_x_at_z_scatter,
    plot_q_vs_qx_at_z_profile,
    plot_eff_sigmoids,
)

plotdir = "/Users/kovacs/cosmology/BulgeDisk/DiskBulgePlots"

In [ ]:
logMz0_min = 8.0
logssfr_min = -16
mass_mask = np.log10(diffstar_cens["smh"][:, -1]) > logMz0_min
sfr_mask = np.log10(diffstar_cens["sSFR"][:, -1]) > logssfr_min
mask = mass_mask & sfr_mask
title = "$M^*_{{z=0}} > 10^{{{:.1f}}} M_\\odot, sSFR > 10^{{{}}} yr^{{-1}}$".format(logMz0_min, logssfr_min)
xname = "log_M0_min_{:.1f}_logssfr_min_{}".format(logMz0_min, -logssfr_min)
print(diffstar_cens["eff_bulge"][mass_mask].shape)

qs = [diffstar_cens['fbulge_early'][mass_mask], diffstar_cens['fbulge_late'][mass_mask]]
ylabels = ['fbulge_early', 'fbulge_late']

In [ ]:
for q, ylabel in zip(qs, ylabels):
    plot_q_vs_q_x_at_z_scatter(q, diffstar_cens['tcrit_bulge'][mass_mask], ylabel, 
              diffstar_cens["eff_bulge"][mass_mask], zindexes, zs,
              cbar_title="$\\epsilon_{bulge}$",
              title=title,
              xname=xname,
              N=2000, plotdir=plotdir,
              xlabel="tcrit_bulge",
              plotsubdir='Fbulge',
              pltname="{}_vs_tcrit_bulge_{}.png",
)

In [ ]:
plot_q_vs_q_x_at_z_scatter(diffstar_cens['fbulge_late'][mass_mask], diffstar_cens['fbulge_early'][mass_mask], 'fbulge_late', 
              diffstar_cens["eff_bulge"][mass_mask], zindexes, zs,
              cbar_title="$\\epsilon_{bulge}$",
              title=title,
              xname=xname,
              N=2000, plotdir=plotdir,
              plotsubdir='Fbulge',
              xlabel="fbulge_early",
              pltname="{}_vs_fbulge_early_{}.png",
)

In [ ]:
Df = diffstar_cens['fbulge_early'][mass_mask] - diffstar_cens['fbulge_late'][mass_mask]
plot_q_vs_q_x_at_z_scatter(Df, diffstar_cens['tcrit_bulge'][mass_mask],
                           'dfbulge', 
              diffstar_cens["eff_bulge"][mass_mask], zindexes, zs,
              cbar_title="$\\epsilon_{bulge}$",
              title=title,
              xname=xname,
              N=2000, plotdir=plotdir,
              xlabel="tcrit_bulge",
              plotsubdir='Fbulge',
              pltname="{}_vs_tcrit_bulge_{}.png",
)

In [ ]:
#eff_bulge = _bulge_fraction_vs_tform(tarr, t10, t90, fbulge_params)
#        dt = t90 - t10
#          = _bulge_fraction_kernel(tarr, fbulge_tcrit, fbulge_early, fbulge_late, dt)
#          
#        tw_h = dt / 6.0
# _bulge_fraction_kernel(t, thalf, frac_early, frac_late, dt) = _tw_sigmoid(t, thalf, tw_h, frac_early, frac_late)
# 
# Putting call sequence together:
#        eff_bulge = _tw_sigmoid(tarr, fbulge_tcrit, dt/6, fbulge_early, fbulge_late)
# 
# tri-weight sigmoid is approximation to sigmoid 
# tri-wt sigmoid =   _tw_sigmoid(x, x0, tw_h, ymin, ymax)
# cf  _sigmoid(x, x0, k, ymin, ymax)

In [ ]:
#Try an example to see how fbulge_params determine eff_bulge
# Look at t10, t90 and dt distributions
tarr = diffstar_cens["t_table"]
smh_pop = diffstar_cens["smh"][mass_mask]
t10 = calc_tform_pop(tarr, smh_pop, 0.1)
t90 = calc_tform_pop(tarr, smh_pop, 0.9)
dt = (t90 - t10)
tcrit = diffstar_cens["tcrit_bulge"][mass_mask]
early = diffstar_cens["fbulge_early"][mass_mask]
late = diffstar_cens["fbulge_late"][mass_mask]
print(t90.shape, dt.shape, tcrit.shape, early.shape)

In [ ]:
qs = [t10, t90, dt, tcrit, early, late]
ylabels = ['t10', 't90', 'dt', 'fbulge_{tcrit}', 'fbulge_{early}', 'fbulge_{late}']
logM_min = 7.0
logM_max = 11.5
Nm = 36
Mbins = np.linspace(logM_min, logM_max, Nm+1)
#print(Mbins)

In [ ]:
plot_q_vs_qx_at_z_profile(qs, jnp.log10(diffstar_cens['smh'])[mass_mask], ylabels, 
              zindexes, zs, bins=Mbins,
              title=title,
              xname=xname,
              plotdir=plotdir,
              pltname="t10_90_dt_fbulge_vs_log_Mz_{}.png",
              plotsubdir='Fbulge',
)

In [ ]:
logsSFR_min = -16
logsSFR_max = -8
Ns = 32
sbins = np.linspace(logsSFR_min, logsSFR_max, Ns +1)

plot_q_vs_qx_at_z_profile(qs, jnp.log10(diffstar_cens['sSFR'])[mass_mask], ylabels, 
              zindexes, zs, bins=sbins,
              title=title, 
              xname=xname, xlabel='$\\log_{10}(sSFR*yr)$',
              plotdir=plotdir,
              plotsubdir='Fbulge',
              pltname="t10_90_dt_fbulge_vs_log_sSFR_{}.png",
              colors=('b', 'g', 'c', 'r', 'darkorange', 'm'),
)

In [ ]:
import itertools
FBULGE_MIN = 0.25
FBULGE_MAX = 0.85
Npts = 3
Nt = 3
f_e_init =  np.linspace(FBULGE_MIN, FBULGE_MAX, Npts+1)
f_tcrit = np.linspace(1.0, 13.0, Nt+1)
Dt = np.linspace(2., 8., Nt-1)
Df_max=0.5
DF = np.linspace(0.05, Df_max, 3)
# generate pairs of typical f_early and f_late values
f_late=[]
f_early=[]
for e in f_e_init:
    late = np.asarray([(e - df) for df in DF if e-df > 0])
    early = np.ones(len(late))*e 
    f_late.append(late)
    f_early.append(early)

f_late = np.asarray(list(itertools.chain.from_iterable(f_late)))
f_early = np.asarray(list(itertools.chain.from_iterable(f_early)))
print(f_late)
print(f_early)

In [ ]:
plot_eff_sigmoids(tarr, f_tcrit, f_early, f_late, Dt, plotdir=plotdir,
                 )